In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import re

# Median Income

In [2]:
income='https://www.payscale.com/college-salary-report/bachelors?page=101'

In [3]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
driver =  webdriver.Chrome(chromedriver)
driver.get(income)
time.sleep(1);

In [5]:
full_list = driver.find_element_by_link_text('See Full List')
full_list.click()
time.sleep(10);

In [6]:
table=driver.find_element_by_class_name('table')

In [7]:
table_html=table.get_attribute('innerHTML')

In [8]:
rank=pd.read_html(table_html)

In [9]:
def isodd(n):
    if n%2==0:
        return False
    return True

In [10]:
df_income=rank[1]

In [11]:
for i in range(2,len(rank)):
    if isodd(i):
        df_income=df_income.merge(rank[i], on=0)

In [12]:
df_income.head()

,0,1_x,1_y,1_x,1_y,1_x,1_y,1_x,1_y,1_x,...,1_x,1_y,1_x,1_y,1_x,1_y,1_x,1_y,1_x,1_y
0,School Type,"Engineering, Liberal Arts School, Private School","Ivy League, Private School, Research Universit...","Engineering, Private School, Research University",State School,"Liberal Arts School, Sober School, For Sports ...","Engineering, Liberal Arts School, Sober School...","Engineering, Private School, Research University","Business, Private School","Ivy League, Private School, Research Universit...",...,"Private School, Religious",Private School,"Private School, Religious","Private School, Religious","Private School, Religious","Liberal Arts School, Private School, Religious","Liberal Arts School, Private School, Religious","Private School, Religious","For Sports Fans, State School","Private School, Religious"
1,Degree Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,School Sector,Private not-for-profit,Private not-for-profit,Private not-for-profit,Public,Public,Public,Private not-for-profit,Private not-for-profit,Private not-for-profit,...,Private not-for-profit,Private not-for-profit,Private not-for-profit,Private not-for-profit,Private not-for-profit,Private not-for-profit,Private not-for-profit,Private not-for-profit,Public,Private not-for-profit
3,Early Career Pay,"$81,000","$69,800","$81,500","$71,600","$75,100","$75,000","$78,800","$66,600","$69,200",...,"$34,100","$35,600","$35,500","$33,700","$34,100","$35,100","$34,700","$36,600","$33,400","$33,300"
4,Mid-Career Pay,"$155,800","$147,800","$147,000","$145,100","$144,300","$143,800","$142,500","$141,700","$140,700",...,"$53,300","$52,700","$52,100","$51,700","$51,600","$51,500","$51,300","$51,200","$51,000","$49,800"
5,% High Meaning,58%,44%,53%,52%,65%,64%,52%,34%,56%,...,NaN,NaN,NaN,NaN,56%,79%,NaN,71%,69%,72%
6,% STEM Degrees,85%,47%,69%,35%,43%,60%,97%,0%,19%,...,10%,0%,0%,0%,18%,18%,29%,7%,10%,0%


In [13]:
df_income=df_income.transpose()
df_income.columns = df_income.iloc[0]

In [14]:
df_income=df_income.drop(0)
df_income.head()

,School Type,Degree Type,School Sector,Early Career Pay,Mid-Career Pay,% High Meaning,% STEM Degrees
1_x,"Engineering, Liberal Arts School, Private School",NaN,Private not-for-profit,"$81,000","$155,800",58%,85%
1_y,"Ivy League, Private School, Research Universit...",NaN,Private not-for-profit,"$69,800","$147,800",44%,47%
1_x,"Engineering, Private School, Research University",NaN,Private not-for-profit,"$81,500","$147,000",53%,69%
1_y,State School,NaN,Public,"$71,600","$145,100",52%,35%
1_x,"Liberal Arts School, Sober School, For Sports ...",NaN,Public,"$75,100","$144,300",65%,43%


In [15]:
schools=[]
for i in range(0,len(rank)):
    if not isodd(i):
        schools.append(str(rank[i][1][0]))
df_income['name']=schools
df_income=df_income.reset_index(drop=True)
df_income.head()

# LAC Ranking

In [19]:
lib_arts='https://www.usnews.com/best-colleges/rankings/national-liberal-arts-colleges?_mode=table'

In [20]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [21]:
driver =  webdriver.Chrome(chromedriver)
driver.get(lib_arts)
time.sleep(1);

In [22]:
for i in range(30):
    try:
        load_more = driver.find_element_by_link_text('Load More')
        load_more.click()
    except:
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
        driver.execute_script("window.scrollBy(0,-1250)")

In [23]:
lac=driver.find_element_by_class_name('search-results-view')

In [24]:
lac_html=lac.get_attribute('innerHTML')

In [25]:
lac_ranks=pd.read_html(lac_html)

In [26]:
lac_final=lac_ranks[0]
lac_final.tail()

In [28]:
school_info=[]
for value in lac_final['Name/Rank']:
    try:
        school_info.append(value.replace('in National Liberal Arts Colleges', ''))
    except:
        pass
        school_info.append('na')

In [29]:
school_info2=[]
for value in school_info:
    try:
        a=re.split(',', value)
        a[0] = [a[0].split(' ')]
        b = str(a[0][0][-1])
        a[0] = str(' '.join(a[0][0][:-1]))
        a[1:2]=a[1].split('#')
        a[1:2]=a[1].split(',')
        a.insert(1,b)
        school_info2.append(a)
    except:
        pass

In [30]:
school_info_dict={'name':[], 'city':[], 'state':[], 'rank':[]}

In [31]:
for value in school_info2:
    school_info_dict['name'].append(value[0])
    school_info_dict['city'].append(value[1])
    school_info_dict['state'].append(value[2])
    school_info_dict['rank'].append(value[-1])

In [32]:
school_infodf=pd.DataFrame(school_info_dict)
lac_final=lac_final.dropna(how='all')
lac_final=lac_final.reset_index()
lac_final=lac_final.join(school_infodf)
lac_final=lac_final.drop(columns='Name/Rank')

In [37]:
# lac_final=lac_final.drop(columns=['level_0', 'index'])

In [38]:
fees=lac_final['Tuition and Fees'].str.replace(",", "")
fees=fees.str.replace("$", "")

In [39]:
in_state=[]
tuition=[]
for prices in fees:
    if type(prices)==str:
        if 'out' in prices:
            tuition.append(prices.split('(out-of-state)')[0])
            in_state.append(prices.split('(out-of-state)')[1])
        else:
            tuition.append(prices)
            in_state.append('NaN')
    else:
        in_state.append('NaN')

In [40]:
in_state=pd.Series(in_state)
tuition=pd.Series(tuition)

In [41]:
in_state=in_state.str.strip('(in-state)').str.strip(' ')

In [42]:
tuition=tuition.str.strip(' ')

In [43]:
lac_final['base_fees']=tuition
lac_final['in-state']=in_state

,index,Tuition and Fees,Enrollment,? Test Scores,Compare,name,city,state,rank,base_fees,in-state
0,0,"$53,550",2076.0,NaN,NaN,Williams College,Williamstown,MA,1,53550,NaN
1,1,"$54,310",1849.0,NaN,NaN,Amherst College,Amherst,MA,2,54310,NaN
2,2,"$51,848",1806.0,NaN,NaN,Bowdoin College,Brunswick,ME,3 (tie),51848,NaN
3,3,"$50,822",1543.0,NaN,NaN,Swarthmore College,Swarthmore,PA,3 (tie),50822,NaN
4,4,"$51,148",2347.0,NaN,NaN,Wellesley College,Wellesley,MA,3 (tie),51148,NaN


In [44]:
lac_final=lac_final.drop(columns=['Tuition and Fees', '?  Test Scores', 'Compare'])

In [45]:
lac_final.head()

,index,Enrollment,name,city,state,rank,base_fees,in-state
0,0,2076.0,Williams College,Williamstown,MA,1,53550,NaN
1,1,1849.0,Amherst College,Amherst,MA,2,54310,NaN
2,2,1806.0,Bowdoin College,Brunswick,ME,3 (tie),51848,NaN
3,3,1543.0,Swarthmore College,Swarthmore,PA,3 (tie),50822,NaN
4,4,2347.0,Wellesley College,Wellesley,MA,3 (tie),51148,NaN


In [60]:
lac_final.index[lac_final['name'] == "Wheaton College"].tolist()

[63, 80]

In [61]:
lac_final.iloc[63]

index                      75
Enrollment               2456
name          Wheaton College
city                  Wheaton
state                     IL 
rank               63   (tie)
base_fees               35190
in-state                  NaN
Name: 63, dtype: object

In [62]:
lac_final.iloc[80]

index                      96
Enrollment               1651
name          Wheaton College
city                   Norton
state                     MA 
rank               76   (tie)
base_fees               45970
in-state                  NaN
Name: 80, dtype: object

In [ ]:
lac_final['name']=lac_final.name.str.strip(" ")

In [72]:
lac_final['rank']=lac_final['rank'].str.strip(' ').str.strip('(tie)').str.strip(' ')

# Read at CSV

In [73]:
df_income.head()

,School Type,Degree Type,School Sector,Early Career Pay,Mid-Career Pay,% High Meaning,% STEM Degrees,name
0,"Engineering, Liberal Arts School, Private School",NaN,Private not-for-profit,"$81,000","$155,800",58%,85%,Harvey Mudd College
1,"Ivy League, Private School, Research Universit...",NaN,Private not-for-profit,"$69,800","$147,800",44%,47%,Princeton University
2,"Engineering, Private School, Research University",NaN,Private not-for-profit,"$81,500","$147,000",53%,69%,Massachusetts Institute of Technology
3,State School,NaN,Public,"$71,600","$145,100",52%,35%,SUNY Maritime College
4,"Liberal Arts School, Sober School, For Sports ...",NaN,Public,"$75,100","$144,300",65%,43%,United States Military Academy


In [75]:
df_income.to_csv('income.csv')

In [74]:
lac_final.head()

,index,Enrollment,name,city,state,rank,base_fees,in-state
0,0,2076.0,Williams College,Williamstown,MA,1,53550,NaN
1,1,1849.0,Amherst College,Amherst,MA,2,54310,NaN
2,2,1806.0,Bowdoin College,Brunswick,ME,3,51848,NaN
3,3,1543.0,Swarthmore College,Swarthmore,PA,3,50822,NaN
4,4,2347.0,Wellesley College,Wellesley,MA,3,51148,NaN


In [76]:
lac_final.to_csv('lac_ranking.csv')